In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Dell\\Documents\\Brain-Tumor-Classification-Project'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from braintumorClassifier.constants import *
from braintumorClassifier.utils.common import read_yaml, create_directories, save_json

[2024-02-10 16:45:55,427: INFO: __init__: Starting data processing]
[2024-02-10 16:45:55,428: INFO: __init__: Data preprocessing completed successfully.]


In [9]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAM_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])


    def get_validation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model='artifacts/training/model.h5',
                training_data='artifacts/data_ingestion/data',
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE,
          )
          return eval_config
          

In [10]:
from urllib.parse import urlparse

In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2,
            zoom_range = 0.2,
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = 'bilinear',

        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs

        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.models.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)


    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path('scores.json'), data=scores)


In [12]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    eval = Evaluation(val_config)
    eval.evaluation()
    eval.save_score()
except Exception as e:
    raise e

[2024-02-10 16:45:55,657: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-10 16:45:55,662: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-10 16:45:55,664: INFO: common: created directory at: artifacts]
Found 208 images belonging to 2 classes.

[2024-02-10 16:46:00,096: WARNING: module_wrapper: From c:\Users\Dell\anaconda1\envs\Project1\Lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
13/13 [==============================] - 14s 891ms/step - loss: 25.7295 - accuracy: 0.8413
[2024-02-10 16:46:12,810: INFO: common: json file: scores.json saved successfully]
